<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# 3.4.5 Evaluation

## Lesson Objectives

After completing this lesson you should be able to:

* Evaluate binary classification algorithms using area under the Receiver Operating Characteristic (ROC) curve
* Evaluate multiclass classification and regression algorithms using several metrics
* Evaluate logistic and linear regression algorithms using summaries

##Evaluators

After training a model and making predictions for the test data it is time to evaluate the model.
* An evaluator is a class that computes metrics from the predictions
* There are three types of evaluators available:
  * `BinaryClassificationEvaluator`
  * `MultiClassClassificationEvaluator`
  * `RegressionEvaluator` 

## Continuing from previous example

If you haven't downloaded the data set from the previous lesson then there is a link in the script to download it to your temporary folder and load it.

In [ ]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

In [ ]:
import org.apache.spark.mllib.util.MLUtils.{
  convertVectorColumnsFromML => fromML,
  convertVectorColumnsToML => toML
}

In [ ]:
import org.apache.spark.mllib.util.MLUtils
 
val data = toML(MLUtils.loadLibSVMFile(sc, "/resources/data/sample_libsvm_data.txt").toDF())

val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

## Example of Logistic Regression

Now we look at an example of binary classification using Logistic Regression. First I create a new instance of a Logistic Regression and set its parameters:

* The maximum number of iterations
* Regularization
* Elastic Net

In [ ]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.classification.BinaryLogisticRegressionSummary

val logr = new LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

val logrModel = logr.fit(trainingData)

println(s"Weights: ${logrModel.coefficients} Intercept: ${logrModel.intercept}")

summary

In [ ]:
logrModel.summary.objectiveHistory

## BinaryClassificationEvaluator

Let's start with the `BinaryClassificationEvaluator`:

* Evaluator for binary classification
* Expects two input columns: **rawPrediction** and **label**
* Supported metric: `areaUnderROC`

As its name states, it is used to evaluate binary classifiers. It expects two input columns, the `rawPrediction` column and the label column. The only supported metric is the area under the ROC curve.

This is an example of a Binary Classification Evaluator. I'm going to build upon the Logistic Regression model from the previous lesson and evaluate its predictions. First, I call the `transform` method on the test data to get a `DataFrame` with the predictions, which I name `predictionsLogR`:

In [ ]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val predictionsLogR = logrModel.transform(testData)

Then, I create a new instance of a `BinaryClassificationEvaluator` and set the corresponding columns as inputs and the metric name to the only available metric, `areaUnderROC`:

In [ ]:
val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("rawPrediction").setMetricName("areaUnderROC")

Now I can call the evaluator's evaluate method on the predictions made by the Logistic Regression to get its area under the ROC curve:

In [ ]:
val roc = evaluator.evaluate(predictionsLogR)

## MulticlassClassificationEvaluator

Often, there are more than two categories you can classify an item into. The Multi-Class Classification Evaluator is an evaluator for multi-class classification problems.

* Expects two input columns: **prediction** and **label**
* Supported metrics:
  * `F1` (default)
  * Precision
  * Recall
  * `weightedPrecision`
  * `weightedRecall`

## Reusing RF Classification Example I

To show what a `Multiclass` Classification Evaluator can do we will need a model that can do more than the two categories the Random Forest classifier we calculated before would do. We will need to prepare the Pipeline for that model.
This is the exact script we have run in previous sessions to set up Pipelines for Random Forests and Gradient-Boosting Trees:

In [ ]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.{StringIndexer, IndexToString, VectorIndexer}

val labelIndexer = new StringIndexer().setInputCol("label").setOutputCol("indexedLabel").fit(data)
val labelConverter = new IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(labelIndexer.labels)
val featureIndexer = new VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(4).fit(data)

In [ ]:
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel

val rfC = new RandomForestClassifier().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures").setNumTrees(3)

## Reusing RF Classification Example II

In [ ]:
import org.apache.spark.ml.Pipeline

// split into training and test data
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

val pipelineRFC = new Pipeline().setStages(Array(labelIndexer, featureIndexer, rfC, labelConverter))
val modelRFC = pipelineRFC.fit(trainingData)
val predictionsRFC = modelRFC.transform(testData)

All the rest is exactly the same as before, calling the `fit` method to get a model
and calling the `transform` method to make predictions. The predictions are then
returned in the `predictionsRFC` `DataFrame`.

## MulticlassClassificationEvaluator

Now an example of a Multi Class Evaluator. For this example, I can evaluate any of the multiclass classifiers I have trained so far, and I choose to evaluate the predictions made by the Random Forest Classifier, which I previously assigned to the `predictionsRFC` `DataFrame`.

The true labels of the test set were in the indexed label column and the predictions made by the model were in its prediction column. So, I create a new instance of a `MulticlassClassificationEvaluator` and set the corresponding columns as inputs. Also, I set the metric to be **precision** instead of the default **F1-score**.

In [ ]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

val evaluator = new MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction").setMetricName("accuracy") 
val accuracy = evaluator.evaluate(predictionsRFC)

println("Test Error = " + (1.0 - accuracy))

Now I can call the evaluator's evaluate method on the predictions made by the Random Forest Classifier to get the estimated precision, which is 96.6% or, put in another way, a 3.3% test error.

## RegressionEvaluator

* Evaluator for regression problems
* Expects two input columns: **prediction** and **label**
* Supported metrics:
  * **rmse**: root mean squared error (default)
  * **mse**: mean squared error
  * **r2**: R2, the coefficient of determination
  * **mae**: mean absolute error

## Reusing RF Regression Example

We will use the previous regression in our previous lesson in Random Forest. If you've come to this lesson directly and don't have the context, here is the code that produces the predictions we will evaluate:

In [ ]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.regression.RandomForestRegressionModel

val rfR = new RandomForestRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures")
val pipelineRFR = new Pipeline().setStages(Array(featureIndexer, rfR))
val modelRFR = pipelineRFR.fit(trainingData)
val predictions = modelRFR.transform(testData)

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.